In [ ]:
import pandas as pd
df= pd.read_excel('dataset-uci 2.xls')

In [ ]:
import pandas as pd

#데이터 읽기
df= pd.read_excel("dataset-uci 2.xls")

# 23번째 데이터 삭제（index=22)
df= df.drop(index=23)

   Gallstone Status  Age  Gender  Comorbidity  Coronary Artery Disease (CAD)  \
0                 0   50       0            0                              0   
1                 0   47       0            1                              0   
2                 0   61       0            0                              0   
3                 0   41       0            0                              0   
4                 0   42       0            0                              0   

   Hypothyroidism  Hyperlipidemia  Diabetes Mellitus (DM)  Height  Weight  \
0               0               0                       0     185    92.8   
1               0               0                       0     176    94.5   
2               0               0                       0     171    91.1   
3               0               0                       0     168    67.7   
4               0               0                       0     178    89.6   

   ...  High Density Lipoprotein (HDL)  Triglyceride  \


In [ ]:
selected_features = ['Age', 'Gender', 'Comorbidity', 'Coronary Artery Disease (CAD)',
                     'Hypothyroidism', 'Hyperlipidemia', 'Diabetes Mellitus (DM)', 'Weight',
                     'Extracellular Fluid/Total Body Water (ECF/TBW)', 'Body Protein Content (Protein) (%)',
                     'Hepatic Fat Accumulation (HFA)', 'Glucose', 'Total Cholesterol (TC)', 'High Density Lipoprotein (HDL)',
                     'Triglyceride', 'Aspartat Aminotransferaz (AST)', 'Alkaline Phosphatase (ALP)', 'Creatinine', 'Glomerular Filtration Rate (GFR)',
                     'C-Reactive Protein (CRP)', 'Hemoglobin (HGB)', 'Vitamin D']

X_selected = df[selected_features].values
y = df.iloc[:, 0].values

# 7:3으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=0, stratify=y)

# 정규화
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(random_state=1, max_iter=1000,C=100)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_train_std)
print(classification_report(y_train, y_pred))

#AUC계산
from sklearn.metrics import roc_auc_score
y_prob = model.predict_proba(X_train_std)[:, 1]
auc = roc_auc_score(y_train, y_prob)
print(f"AUC: {auc:.4f}")

# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(random_state=1, max_iter=1000,C=100)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
print(classification_report(y_test, y_pred))

#AUC계산
from sklearn.metrics import roc_auc_score
y_prob = model.predict_proba(X_test_std)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print(f"AUC: {auc:.4f}")


              precision    recall  f1-score   support

           0       0.83      0.90      0.87       112
           1       0.89      0.82      0.85       110

    accuracy                           0.86       222
   macro avg       0.86      0.86      0.86       222
weighted avg       0.86      0.86      0.86       222

AUC: 0.9393
              precision    recall  f1-score   support

           0       0.73      0.77      0.75        48
           1       0.76      0.71      0.73        48

    accuracy                           0.74        96
   macro avg       0.74      0.74      0.74        96
weighted avg       0.74      0.74      0.74        96

AUC: 0.7858


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score

# 데이터 준비
selected_features = ['Age', 'Gender', 'Comorbidity', 'Coronary Artery Disease (CAD)',
                     'Hypothyroidism', 'Hyperlipidemia', 'Diabetes Mellitus (DM)', 'Weight',
                     'Extracellular Fluid/Total Body Water (ECF/TBW)', 'Body Protein Content (Protein) (%)',
                     'Hepatic Fat Accumulation (HFA)', 'Glucose', 'Total Cholesterol (TC)', 'High Density Lipoprotein (HDL)',
                     'Triglyceride', 'Aspartat Aminotransferaz (AST)', 'Alkaline Phosphatase (ALP)', 'Creatinine',
                     'Glomerular Filtration Rate (GFR)', 'C-Reactive Protein (CRP)', 'Hemoglobin (HGB)', 'Vitamin D']

X = df[selected_features].values
y = df.iloc[:, 0].values  # 타겟이 첫 번째 열이라고 가정

# Stratified K-Fold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 결과 저장용
auc_scores = []
accuracy_scores = []
fold = 1

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # 정규화 (각 fold 마다 따로 해야 함)
    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)

    # 로지스틱 회귀
    model = LogisticRegression(C=100, random_state=1, max_iter=1000)
    model.fit(X_train_std, y_train)

    y_pred = model.predict(X_test_std)
    y_proba = model.predict_proba(X_test_std)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = model.score(X_test_std, y_test)

    auc_scores.append(auc)
    accuracy_scores.append(acc)

    print(f"\n📂 Fold {fold}")
    print(classification_report(y_test, y_pred))
    print(f"AUC: {auc:.4f}, Accuracy: {acc:.4f}")
    fold += 1

# 평균 성능
print("\n✅ K-Fold 결과 요약:")
print(f"평균 AUC: {np.mean(auc_scores):.4f}")
print(f"평균 정확도: {np.mean(accuracy_scores):.4f}")



📂 Fold 1
              precision    recall  f1-score   support

           0       0.69      0.84      0.76        32
           1       0.80      0.62      0.70        32

    accuracy                           0.73        64
   macro avg       0.75      0.73      0.73        64
weighted avg       0.75      0.73      0.73        64

AUC: 0.8174, Accuracy: 0.7344

📂 Fold 2
              precision    recall  f1-score   support

           0       0.74      0.88      0.80        32
           1       0.85      0.69      0.76        32

    accuracy                           0.78        64
   macro avg       0.79      0.78      0.78        64
weighted avg       0.79      0.78      0.78        64

AUC: 0.8125, Accuracy: 0.7812

📂 Fold 3
              precision    recall  f1-score   support

           0       0.85      0.72      0.78        32
           1       0.76      0.88      0.81        32

    accuracy                           0.80        64
   macro avg       0.80      0.80     

In [ ]:
selected_features = ['Age', 'Gender', 'Comorbidity', 'Coronary Artery Disease (CAD)',
                     'Hypothyroidism', 'Hyperlipidemia', 'Diabetes Mellitus (DM)', 'Weight',
                     'Extracellular Fluid/Total Body Water (ECF/TBW)', 'Body Protein Content (Protein) (%)',
                     'Hepatic Fat Accumulation (HFA)', 'Glucose', 'Total Cholesterol (TC)', 'High Density Lipoprotein (HDL)',
                     'Triglyceride', 'Aspartat Aminotransferaz (AST)', 'Alkaline Phosphatase (ALP)', 'Creatinine', 'Glomerular Filtration Rate (GFR)',
                     'C-Reactive Protein (CRP)', 'Hemoglobin (HGB)', 'Vitamin D']

X_selected = df[selected_features].values
y = df.iloc[:, 0].values

# 7:3으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=0, stratify=y)

# 정규화
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(C=1.0, penalty='l2', solver='liblinear',
                           random_state=1, max_iter=1000)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_train_std)
print(classification_report(y_train, y_pred))

#AUC계산
from sklearn.metrics import roc_auc_score
y_prob = model.predict_proba(X_train_std)[:, 1]
auc = roc_auc_score(y_train, y_prob)
print(f"AUC: {auc:.4f}")

# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(C=1.0, penalty='l2', solver='liblinear',
                           random_state=1, max_iter=1000)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
print(classification_report(y_test, y_pred))

#AUC계산
from sklearn.metrics import roc_auc_score
y_prob = model.predict_proba(X_test_std)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print(f"AUC: {auc:.4f}")


              precision    recall  f1-score   support

           0       0.84      0.91      0.88       112
           1       0.90      0.83      0.86       110

    accuracy                           0.87       222
   macro avg       0.87      0.87      0.87       222
weighted avg       0.87      0.87      0.87       222

AUC: 0.9352
              precision    recall  f1-score   support

           0       0.76      0.79      0.78        48
           1       0.78      0.75      0.77        48

    accuracy                           0.77        96
   macro avg       0.77      0.77      0.77        96
weighted avg       0.77      0.77      0.77        96

AUC: 0.7943


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score

# 데이터 준비
selected_features = ['Age', 'Gender', 'Comorbidity', 'Coronary Artery Disease (CAD)',
                     'Hypothyroidism', 'Hyperlipidemia', 'Diabetes Mellitus (DM)', 'Weight',
                     'Extracellular Fluid/Total Body Water (ECF/TBW)', 'Body Protein Content (Protein) (%)',
                     'Hepatic Fat Accumulation (HFA)', 'Glucose', 'Total Cholesterol (TC)', 'High Density Lipoprotein (HDL)',
                     'Triglyceride', 'Aspartat Aminotransferaz (AST)', 'Alkaline Phosphatase (ALP)', 'Creatinine',
                     'Glomerular Filtration Rate (GFR)', 'C-Reactive Protein (CRP)', 'Hemoglobin (HGB)', 'Vitamin D']

X = df[selected_features].values
y = df.iloc[:, 0].values  # 타겟이 첫 번째 열이라고 가정

# Stratified K-Fold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 결과 저장용
auc_scores = []
accuracy_scores = []
fold = 1

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # 정규화 (각 fold 마다 따로 해야 함)
    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)

    # 로지스틱 회귀
    model = LogisticRegression(C=1, random_state=1, max_iter=1000)
    model.fit(X_train_std, y_train)

    y_pred = model.predict(X_test_std)
    y_proba = model.predict_proba(X_test_std)[:, 1]

    auc = roc_auc_score(y_test, y_proba)
    acc = model.score(X_test_std, y_test)

    auc_scores.append(auc)
    accuracy_scores.append(acc)

    print(f"\n📂 Fold {fold}")
    print(classification_report(y_test, y_pred))
    print(f"AUC: {auc:.4f}, Accuracy: {acc:.4f}")
    fold += 1

# 평균 성능
print("\n✅ K-Fold 결과 요약:")
print(f"평균 AUC: {np.mean(auc_scores):.4f}")
print(f"평균 정확도: {np.mean(accuracy_scores):.4f}")



📂 Fold 1
              precision    recall  f1-score   support

           0       0.69      0.84      0.76        32
           1       0.80      0.62      0.70        32

    accuracy                           0.73        64
   macro avg       0.75      0.73      0.73        64
weighted avg       0.75      0.73      0.73        64

AUC: 0.8193, Accuracy: 0.7344

📂 Fold 2
              precision    recall  f1-score   support

           0       0.74      0.91      0.82        32
           1       0.88      0.69      0.77        32

    accuracy                           0.80        64
   macro avg       0.81      0.80      0.79        64
weighted avg       0.81      0.80      0.79        64

AUC: 0.8066, Accuracy: 0.7969

📂 Fold 3
              precision    recall  f1-score   support

           0       0.88      0.69      0.77        32
           1       0.74      0.91      0.82        32

    accuracy                           0.80        64
   macro avg       0.81      0.80     